In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math as mth
import scipy.stats as stats
from scipy import stats as st
from plotly import graph_objects as go
from datetime import datetime


In [ ]:
import plotly.io as pio
pio.renderers.default='notebook'

# Тема: оценка результатов проведения А/В теста

**Описание данных**

`ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год.

Структура файла:
* `name` — название маркетингового события;
* `regions` — регионы, в которых будет проводиться рекламная кампания;
* `start_dt` — дата начала кампании;
* `finish_dt` — дата завершения кампании.

`final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.
 
Структура файла:
* `user_id` — идентификатор пользователя;
* `first_date` — дата регистрации;
* `region` — регион пользователя;
* `device` — устройство, с которого происходила регистрация.

`final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

Структура файла:
* `user_id` — идентификатор пользователя;
* `event_dt` — дата и время покупки;
* `event_name` — тип события;
* `details` — дополнительные данные о событии. Например, для покупок, `purchase`, в этом поле хранится стоимость покупки в долларах.


`final_ab_participants.csv` — таблица участников тестов.

Структура файла:

* `user_id` — идентификатор пользователя;
* `ab_test` — название теста;
* `group` — группа пользователя.


 ## Исследование данных 

In [ ]:
events = pd.read_csv('/datasets/final_ab_events.csv')
new_users = pd.read_csv('/datasets/final_ab_new_users.csv')
participants = pd.read_csv('/datasets/final_ab_participants.csv')
project_marketing_events = pd.read_csv('/datasets/ab_project_marketing_events.csv')


In [ ]:
frame = {"ab_events":events,
         "ab_new_users":new_users,
         "ab_participants":participants,
         "marketing_events":project_marketing_events
        }  #создадим словарь для того, чтобы можно было по нему пробежаться

for name, information in frame.items():
    print('\033[1m' + name.upper() + '\033[0m')
    display(information.head()) #открываем фрейм
    print()
    print(information.info())   #выводим информацию о фрейме
    print()
    print("Количество пропусков по столбцам")
    print(information.isna().sum()) #посмотрим на пропуски
    print()
    print(f'Количество явных дубликатов - {information.duplicated().sum()} ') #выведим явные дубликаты для изучения
    print()

В датасете final_ab_events:
- По столбцу details присутствует 377577 пропусков. Пропуски связаны с тем, что в данном столбце отображается дополнительная информация только о покупках, если покупка не было совершена, то данный столбец не заполняется.
- Столбец event_dt нужно привести к формату datatime

В датасете final_ab_new_users:
- Столбец first_date нужно привести к формату datatime

В датасете final_ab_project_marketing_events:
- Столбец start_dt и finish_dt нужно привести к формату datatime

Приведем даты к формату datatime:

In [ ]:
events['event_dt'] = pd.to_datetime(events['event_dt'], format='%Y-%m-%d %H:%M:%S')
new_users['first_date'] = pd.to_datetime(new_users['first_date'], format='%Y-%m-%d')
project_marketing_events['start_dt'] = pd.to_datetime(project_marketing_events['start_dt'], format='%Y-%m-%d')
project_marketing_events['finish_dt'] = pd.to_datetime(project_marketing_events['finish_dt'], format='%Y-%m-%d')

## Оценка корректности проведения теста

### Соответствие данных требованиям технического задания. 

 **Название теста**: recommender_system_test 

In [ ]:
participants['ab_test'].value_counts()

В данных присутствует помимо нужного нам теста recommender_system_test, еще дополнительный тест interface_eu_test. Сгруппируем по тестам и посмотрим сколько пользоввателей находятся в разных тестах

In [ ]:
participants.groupby(['ab_test', 'group']).agg({'user_id':'nunique'})

Сделаем отдельную переменную  с нужным нам тестом

In [ ]:
recommender_system_test = participants.query('ab_test == "recommender_system_test"')

In [ ]:
recommender_system_test.groupby('group').agg({'user_id':'nunique'})

**Группы**: А — контрольная, B — новая платёжная воронка;

- дата запуска: 2020-12-07;

- дата остановки набора новых пользователей: 2020-12-21;

- дата остановки: 2021-01-04;

In [ ]:
print('Дата запуска:', new_users['first_date'].min())
print('Дата остановки набора новых пользователей:', new_users['first_date'].max())

Попало 2 лишних дня. Срезаем согласно ТЗ.

In [ ]:
new_users = new_users.query('first_date <= "2020-12-21" and region == "EU"') #срезаем по ТЗ

**аудитория**: 15% новых пользователей из региона EU

Посмотрим,  сколько попало в тест из региона EU.

In [ ]:
#присоединям к таблицу участников теста, новых участников из EU
eu_test = recommender_system_test.merge(new_users, on = 'user_id', how = "left")
#дропнем участников, которые не относятся к EU
eu_test.dropna(inplace = True)

In [ ]:
print('Доля пользователей из EU региона попавших в тест: {:.2f}%'.format(len(eu_test)/len(new_users)*100))

Действительно, 15% попадает в тест.

В ТЗ стоит задача улучшить параметры конверсии за 14 дней. В связи с этим, всех пользователей, которые "прожили" меньше удалим из данных для достоверности анализа:

In [ ]:
events_by_user = recommender_system_test.merge(events, on = 'user_id', how = 'left') #сгруппируем участников теста и события

events_by_user = events_by_user.merge(new_users, on = 'user_id', how = 'left') #присоединим к участникам новых пользователей


# вычисляем лайфтайм для каждой сессии в днях

events_by_user['lifetime'] = (
    events_by_user['event_dt'] - events_by_user['first_date']
).dt.days


In [ ]:
result_grouped = events_by_user.pivot_table(
    index=['first_date'], columns='lifetime', values='user_id', aggfunc='nunique'
)

result_grouped

Видим, что пользователи, которые попали в тест после 2020-12-15 не успевают "прожить" 14 дней. Поэтому обрезаем данные по 2020-12-15.

In [ ]:
events_by_user = events_by_user.query('first_date <= "2020-12-15"')

### Время проведения теста. Убедимся, что оно не совпадает с маркетинговыми и другими активностями

Посмотрим на таблицу с маркетинговыми активносями и проверим, выпадают ли они на наше тест. Согласно ТЗ тест проводился в интервале с 7 декабря 2020 по 4 января 2021 года (2020-12-07 по 2021-01-04)

In [ ]:
#посмоотрим на таблицу с маркетинговыми активностями
project_marketing_events.sort_values(by='start_dt').reset_index(drop = True) 

In [ ]:
cross_events = project_marketing_events.query('start_dt <= "2021-01-04" and finish_dt >= "2020-12-07"')
cross_events               

Время теста совпадает с Рождественским и Новогодним промо (Christmas&New Year Promo) и Новогодней лотереей (CIS New Year Gift Lottery). Данные промо, определенно будут влиять на результаты теста.

### Аудитория теста. 

Удостоверимся, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверим равномерность распределения по тестовым группам и правильность их формирования.

Как мы узнали выше, в данных присутствует еще один тест помимо того, который мы изучаем. Посмотрим, пеерсекаются ли пользователи в этих тестах.

In [ ]:
participants.groupby(['ab_test', 'group']).nunique() 

In [ ]:
# Посмотрим пересечение пользователей в тестах.
cross_users = len(participants
                  .groupby('user_id') #сгруппируем по  поьзоваьелям
                  .agg({'ab_test':'nunique'}) #посчитаем уникальные тесты
                  .query('ab_test > 1') #дропнем тех, кто присутсвуют в обоих тестах
               
              )
print(f'Число пользователей попавших в пересекающиеся тесты составляет {cross_users}')

Ранее мы уже фильтровали нужный нам тест в переменной events_by_user. Проверим уже по фильтрованому и проверим не попали ли пользователеи в обе группы:

In [ ]:
events_by_user.groupby(['ab_test', 'group'])['user_id'].nunique() 

Позьзователи по группам распределены не равномерно. Это говорит нам о неоптимальности разбиения данных по группам. Для будущих тестов следует несколько поработать с механикой выделения групп и, возможно, изменить подход к их выделению.

In [ ]:
# Посмотрим пересечение пользователей в тестах.
cross_users_target_test = len(events_by_user
                              .groupby('user_id') #сгруппируем по  поьзоваьелям
                              .agg({'group':'nunique'}) #посчитаем уникальные группы
                              .query('group > 1') #дропнем тех, кто присутсвуют в обоих тестах
                             )
print(f'Число пересекающихся пользователей в группах {cross_users_target_test}')

### Вывод

1) В данные попал еще один тест (interface_eu_test) не указаный в ТЗ , произвели фильтрацию.

2) В данных находятся 2 лишних дня, которые не указаны в ТЗ провели фильрацию.

3) Убрали пользователей, которые не "прожили" 14 дней.

4) Группы разбиты неравномерно.

## Исследовательский анализ данных

### Распределение количества событий на пользователя  в выборках

Посчитаем сколько событий приходится на пользователя.

In [ ]:
#events_by_user = recommender_system_test.merge(events, on = 'user_id', how = 'left') #сгруппируем участников теста и события

In [ ]:
#Выделим отдельно группы
group_A = events_by_user.query('group == "A"')
group_B = events_by_user.query('group == "B"')

In [ ]:
#сгруппируем события и посчитаем количество по тестовым группам
event_by_group = (events_by_user
                  .pivot_table(index = 'event_name', 
                               columns = 'group', 
                               values = 'user_id', 
                               aggfunc = 'count')
                  .reset_index()
                 )
#сгруппируем события по группе А и посчитаем количество уникальных пользователей
a = group_A.groupby('event_name').agg({'user_id':'nunique'}).reset_index() 
#сгруппируем события по группе В и посчитаем количество уникальных пользователей
b = group_B.groupby('event_name').agg({'user_id':'nunique'}).reset_index()

event_by_group['mean_A'] = (event_by_group['A']/a['user_id']).round(2) #добавим среднее событие на пользователя по группе А
event_by_group['mean_B'] = (event_by_group['B']/b['user_id']).round(2) #добавим среднее событие на пользователя по группе В
event_by_group['ratio_mean'] = event_by_group['mean_A']/event_by_group['mean_B']
display(event_by_group)

Среднее количество событий на человека в группе В на 9%-12% меньше чем в группе А. 

Посчитаем среднее количество всех событий на пользователя по группам:

In [ ]:
print('Среднее количество событий на пользователя в группе A:', 
      (event_by_group['A'].sum()/group_A['user_id'].nunique()).round(2))
print('Среднее количество событий на пользователя в группе B:', 
      (event_by_group['B'].sum()/group_B['user_id'].nunique()).round(2))

Среднее количество всех событий на пользователя в группе A в 2 раза больше, чем в группе В.

### Распределение числа событий по дням

Посмотрим на распределение событий по дням.

In [ ]:
events_by_user['day'] = events_by_user['event_dt'].dt.strftime('%Y-%m-%d') #выделим дни

In [ ]:
events_by_day = (events_by_user
                 .pivot_table(index = 'day', 
                              values='event_name', 
                              columns = 'group', 
                              aggfunc = 'count') # сгруппируем количество событий по дням
                 .reset_index()
                )
events_by_day

In [ ]:
plt.figure(figsize = (14,5))

events_by_day.plot(x = 'day', kind = "bar", figsize = (15, 10))

plt.title('Распределение событий по датам')
plt.xlabel('Дата')
plt.ylabel('Количество событий')
plt.grid()

plt.show()


Видно, что распределение по группам неравномерное. С 2020-12-07 по 2020-12-10 пользователи попадали в группы примерно одинаково, однако после группа А стала преобладать над группой В. В 2020-12-14 происходит всплеск в группе А.

### Изменение конверсии в воронке в выборках на разных этапах

Для начала вспомним с какими событиями работаем и расположим их в порядке логического прохождения этапов:
- login - вход
- product_page - просмотр карточек товаров
- product_cart - просмотры корзины	
- purchase - покупка

Посмотрим сколько у нас всего событий сделали пользователи:

In [ ]:
events_funnel = (events_by_user
                  .pivot_table(index = 'event_name', 
                               columns = 'group', 
                               values = 'user_id', 
                               aggfunc = 'count')
                  .reset_index()
                 )
events_funnel #общее количество сделаных событий

Наблюдательно, что просмотривают корзину меньше, чем совершают покупки. Такое может быть, если реализована функция быстрой покупки без просмотра корзины. В связи с эти, конверсию до корзины не будем рассматривать в серьез и отфильтруем ее. 


In [ ]:
events_funnel.query('event_name != "product_cart"', inplace = True)

Посчитаем сколько пользователей совершали каждое из этих событий и  долю пользователей, которые хоть раз совершали событие:

In [ ]:
event_user = (events_by_user
              .query('event_name != "product_cart"') #убираем корзину
              .pivot_table(
                  index = 'event_name',
                  columns = 'group',
                  values = 'user_id',
                  aggfunc = 'nunique' #считаем сколько уникальных пользователей совершали
              )
              .sort_values('A', ascending = False)
              .reset_index()
             )


event_user['ratio_by_user_A'] = (event_user ['A']/group_A['user_id'].nunique())*100 #считаем долю событий к пользователям 
event_user['ratio_by_user_B'] = (event_user ['B']/group_B['user_id'].nunique())*100 #считаем долю событий к пользователям 

event_user

Выведим воронку событий:

In [ ]:
fig = go.Figure()


fig.add_trace(go.Funnel(
    name = 'Group A',
    y = event_user['event_name'],
    x = event_user['A'],
    textinfo = "value+percent initial"
)
)

fig.add_trace(go.Funnel(
    name = 'Group B',
    y = event_user['event_name'],
    x = event_user['B'],
    textinfo = "value+percent initial"
)
)

fig.update_layout(title = 'Воронка событий групп А и В')

fig.show()

In [ ]:
#соберем таблички вместе и добавим конврсию в таблицу
event_count = events_funnel.merge(event_user, on ='event_name', how = 'left')
event_count.rename(columns = ({'A_x': 'total_event_A',
                              'B_x': 'total_event_B',
                              'A_y': 'number_of_users_by_event_A',
                              'B_y': 'number_of_users_by_event_B'}), inplace = True)
event_count ['ratio_by_step_A'] =  event_count['number_of_users_by_event_A']/event_count['number_of_users_by_event_A'].shift(1)
event_count ['ratio_by_step_B'] =  event_count['number_of_users_by_event_B']/event_count['number_of_users_by_event_B'].shift(1)
event_count.fillna(1, inplace = True)

event_count

Соединим таблички с конверсиями пгруппы А и группы В и посмотрим на результаты как изменилась конверсия в группе В относительно группы А:

In [ ]:
total_convers = event_count[['event_name', 'ratio_by_step_A', 'ratio_by_step_B']]
                
total_convers['ratio_by_group'] = (
    total_convers['ratio_by_step_B']/total_convers['ratio_by_step_A']*100-100) 
total_convers

Из таблицы видно, что отношение конверсии между контрольной и тестовой по большей части не является положительной. Конверсия в группе В от страницы **login** до старницы **product_page** упала на 11%. А вот конверсия между **product_page** и **purchase** напротив увеличилась на 10%. 

### Выводы

1) Среднее количество всех событий на пользователя в группе A в 2 раза больше, чем в группе В:

- Среднее количество событий на пользователя в группе A: 3.75
- Среднее количество событий на пользователя в группе B: 1.89

2) Распределение по группам неравномерное. С 2020-12-07 по 2020-12-10 пользователи попадали в группы примерно одинаково, однако после группа А стала преобладать над группой В. В 2020-12-14 происходит всплеск в группе А.

3) Глобального улучшения в конверсии через 14 дней не произошло. Конверсия между login и product_page упала на 11%. А вот product_page и purchase напротив увеличилась на 10%.

4) От логина до покупки в группе А доходит 14,9% пользователей,  в группе В - 15,2%.

5) Корзину просматривают меньше, чем совершают покупки. Скорее всего реализована функция быстрой покупки без просмотра корзины. 

## Оцените результаты A/B-тестирования

### Что можно сказать про результаты A/В-тестирования?

Данное А/В тестировние не корректно по нескольким причинам:

1) На А/В тестирование накладывается 2 промоакции, которые могут искажать события.

2) Деление на группы происходит не равномерно, что сразу бросается в глазах. Деление по группам отличается примерно в 2 раза.

3) Произошло наложение еще одного А/В теста, некоторые пользователи попали в 2 теста сразу.

### Проверим статистическую разницу долей z-критерием.

Проверим статистическую разницу долей:

In [ ]:
event_by_group = (events_by_user    #сгруппируем события по группам по уникальным пользователям
                  .query('event_name != "product_cart"')
                  .pivot_table(
                      index = 'event_name', 
                      columns = 'group',
                      values = 'user_id',
                      aggfunc = 'nunique'
                     )
                  .sort_values(by = 'A', ascending=False)
                  .reset_index()
                 )
event_by_group

In [ ]:
count_users_by_group = (events_by_user
                        .query('event_name != "product_cart"')
                        .groupby('group')['user_id'].nunique()
                       )#посчитаем количество пользователей по группам
count_users_by_group

Сформулируем нулевую и альтернативную гипотезы:

*Нулевая гипотеза H₀*: Между долями нет значительной разности.

*Альтернативная гипотеза H₁*: Между долями есть значительная разность.

Напишем функцию для определения разницы в долях между пользователями на каждом этапе использования сайта и общим количеством пользователей:

In [ ]:
def z_ratio (group1, group2, alpha): #сделаем функцию, чтобы не писать это для кадлого случая
    
    for i in event_by_group.index: #окутаем это все в цикл, чтобы не выводить для каждого события отдельно 
        
        alpha = alpha  # критический уровень статистической значимости
    
        # пропорция успехов в первой группе:
        p1 = event_by_group[group1][i]/count_users_by_group[group1] 
        
        # пропорция успехов во второй группе:
        p2 = event_by_group[group2][i]/count_users_by_group[group2]
        
        # пропорция успехов в комбинированном датасете:
        p_combined = (
            (event_by_group[group1][i] + event_by_group[group2][i]) 
            / (count_users_by_group[group1]  + count_users_by_group[group2] )
        )
        
        # разница пропорций в датасетах
        difference = p1 - p2 

        z_value = difference/mth.sqrt(
            p_combined * (1 - p_combined) * (1/count_users_by_group[group1]  + 1/count_users_by_group[group2] )
        ) 
        
        # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
        distr = st.norm(0, 1)
        

        p_value = (1 - distr.cdf(abs(z_value))) * 2 
        
        print('{} p-значение:{}' .format(event_by_group['event_name'][i], p_value))
        
    
        if p_value < alpha:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        else:
            print(
            'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными'
            )

In [ ]:
z_ratio ('A', 'B', 0.05)

Напишем функцию для определения разницы в долях между числом пользователей на определенном этапе и числом пользователей на предыдущем этапе:

In [ ]:
def z_ratio2 (group1, group2, alpha): #сделаем функцию, чтобы не писать это для кадлого случая
    
    
    for i in event_by_group.index: #окутаем это все в цикл, чтобы не выводить для каждого события отдельно 
        
        try:
            print('Шаг воронки: {} -> {}:'.format(event_by_group['event_name'][i], event_by_group['event_name'][i+1]))
            
            alpha = alpha  # критический уровень статистической значимости
    
            # пропорция успехов в первой группе:
            p1 = event_by_group[group1][i+1]/event_by_group[group1][i]

            # пропорция успехов во второй группе:
            p2 = event_by_group[group2][i+1]/event_by_group[group2][i]

            # пропорция успехов в комбинированном датасете:
            p_combined = (
                (event_by_group[group1][i+1] + event_by_group[group2][i+1]) 
                / (event_by_group[group1][i]  + event_by_group[group2][i] )
            )
            # разница пропорций в датасетах
            difference = p1 - p2 

            z_value = difference/mth.sqrt(
                p_combined * (1 - p_combined) * (1/event_by_group[group1][i]  + 1/event_by_group[group2][i] )
            ) 

            # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
            distr = st.norm(0, 1)
        

            p_value = (1 - distr.cdf(abs(z_value))) * 2 

            print(' p-значение:', p_value.round(2))
        
            if p_value < alpha:
                print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
            
            else:
                print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')
            
        except:
            print('\033[1m  Конец теста   \033[0m')  

In [ ]:
z_ratio2('A', 'B', 0.05)

Между долями групп конверсии:

- login -> product_page - есть значимая разница. 

- product_page -> purchase - нет значимой разницы (доли одинаковые)

Напишем функцию, котрая нам будет сравнивать конверсии между группами от события к покупке (login -> purchase; product_page -> purchase):

In [ ]:
def z_ratio3 (group1, group2, alpha): #сделаем функцию, чтобы не писать это для кадлого случая
    
        
    for i in event_by_group.index: #окутаем это все в цикл, чтобы не выводить для каждого события отдельно
        
            
        print('Шаг воронки: {} -> {}:'.format(event_by_group['event_name'][i], event_by_group['event_name'][2]))
            
        alpha = alpha  # критический уровень статистической значимости
    
         # пропорция успехов в первой группе:
        p1 = event_by_group[group1][i]/event_by_group[group1][2]

         # пропорция успехов во второй группе:
        p2 = event_by_group[group2][i]/event_by_group[group2][2]

         # пропорция успехов в комбинированном датасете:
        p_combined = (
              (event_by_group[group1][2] + event_by_group[group2][2]) 
              / (event_by_group[group1][i]  + event_by_group[group2][i] )
        )

        # разница пропорций в датасетах
        difference = p1 - p2

        z_value = difference/mth.sqrt(
                  p_combined * (1 - p_combined) * (1/event_by_group[group1][2]  + 1/event_by_group[group2][2] )
        ) 

        # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
        distr = st.norm(0, 1)
        

        p_value = (1 - distr.cdf(abs(z_value))) * 2 

        print(' p-значение:', p_value.round(2))
        
        if p_value < alpha:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
            
        else:
            print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')
 

In [ ]:
z_ratio3('A', 'B', 0.05)

Между долями групп конверсии:

- login -> purchase - нет значимой разницы (доли одинаковые) 

- product_page -> purchase - есть значимая разница

###  Вывод

По результатам Z- теста подтвердили, что между долями групп есть значительная статистическая разница.

***После сравнения долей конверсии между группами пришли к выводу***:

- login -> product_page - есть значимая разница.

- product_page -> purchase - нет значимой разницы (доли одинаковые)


***После сравнения долей конверсии событий к покупкам пришли к выводу***:

- login -> purchase - нет значимой разницы (доли одинаковые) 

- product_page -> purchase - есть значимая разница


## Вывод и заключение о корректности проведения теста.

**Общие выводы**

После изучения А/В теста были отмечаны следущие моменты:

1) Согласно ТЗ проведенный тест называется recommender_system_test, однако в данные попал еще один тест interface_eu_test, который не был указан в ТЗ.

2) В данные попали 2 лишних дня, которые не указаны в ТЗ провели фильрацию. Данные дни относятся к другому тесту.

3) Согласно ТЗ 15% новых пользователей из региона EU. После изучения файла, это подтверждается.

4) На А/В тест накладывается 2 промоакции, которые заведомо могут влиять на корректность проведения теста. 

5) Пользователи из целевого теста пересекаются с конкурирующим тестом.

6) Пользователи по группам распределены не равномерно. Это говорит нам о неоптимальности разбиения данных по группам. Для будущих тестов следует несколько поработать с механикой выделения групп и, возможно, изменить подход к их выделению

7) Среднее количество всех событий на пользователя в группе A в 2 раза больше, чем в группе В:

- Среднее количество событий на пользователя в группе A: 3.75
- Среднее количество событий на пользователя в группе B: 1.89

8) Глобального улучшения в конверсии через 14 дней не произошло. Конверсия между login и product_page упала на 11%. А вот product_page и purchase напротив увеличилась на 10%.

9) От логина до покупки в группе А доходит 14,9% пользователей, в группе В - 15,2%.

10) По результатам Z- теста подтвердили, что между долями групп есть значительная статистическая разница.

11) По результатам Z- теста конверсии между группами пришли к выводу:

- login -> product_page - есть значимая разница.

- product_page -> purchase - нет значимой разницы (доли одинаковые)


12) По результатам Z- теста конверсии событий к покупкам пришли к выводу:

- login -> purchase - нет значимой разницы (доли одинаковые) 

- product_page -> purchase - есть значимая разница



**Заключение**

Тест проведен ***некорректно***:

1) На тест влияют наложившиеся промоакции

2) Разбиение на группы происходит некооректно. В группе А значительно больше пользователей, чем в группе В.

3) Тест следует оставновить и признать несостоявшимся
